In [162]:
from collections import defaultdict
import itertools
from matplotlib import pyplot as plt
import math
import numpy as np
import operator
import pandas as pd
import pickle
import random
import re
import scipy
import string
from tqdm import tqdm_notebook


import src.knntools as knntools
import src.fusiontools as fusiontools
import src.postprocess as postprocess
import src.preprocess as preprocess
import src.tfidftools as tfidftools
import src.tools as tools
import src.recencytools as recency
import src.scoring as scoring
import src.textembeddingtools as texttools
import src.graphwordstools as graphtools
import src.manualprocessingtools as manualtools

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path_to_data = 'data/'

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(
    path_to_data + 'training_info.csv', sep=',', parse_dates=True, header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

test_info = pd.read_csv(path_to_data + 'test_info.csv',
                        sep=',', parse_dates=True, header=0)
path_to_results = 'results/'

In [3]:
train_info, train_email_ids_per_sender, val_info, val_email_ids_per_sender = scoring.get_train_val(training, training_info, train_frac=0.95)

Processing training !

Processing val !



In [6]:
train_body_dict = preprocess.body_dict_from_panda(train_info)
val_body_dict = preprocess.body_dict_from_panda(val_info)

In [7]:
all_recipients_in_train = preprocess.get_all_recipients_from_df(train_info)
all_recipients_in_train = [rec.lower() for rec in all_recipients_in_train]

In [78]:
val_body_dict = preprocess.body_dict_from_panda(val_info)

In [80]:
# test_email_ids_per_sender = preprocess.get_email_ids_per_sender(test)

In [81]:
val_mid_sender_dic = preprocess.get_mid_sender_dict(val_email_ids_per_sender)

In [103]:
val_emails_in_content = manualtools.get_filtered_emails_dic(val_body_dict, val_mid_sender_dic, candidate_list=None)

In [254]:
train_mid_sender_dic = preprocess.get_mid_sender_dict(train_email_ids_per_sender)
train_emails_in_content = manualtools.get_filtered_emails_dic(train_body_dict, train_mid_sender_dic, candidate_list=all_recipients_in_train)

In [190]:

current_score = scoring.compute_prediction_mad(in_body_predictions, val_info)
print(current_score)

2181
0.00313311936421


In [106]:
with open('variables/emails_in_val', 'wb') as outfile:
    pickle.dump(emails_in_content, outfile)

In [243]:
extracted_subject_length = 10
keyword = 'From:'
val_subject_dic = manualtools.get_keyword_dic(val_body_dict, keyword=keyword,
                                              extracted_length=extracted_subject_length)
train_subject_dic = manualtools.get_keyword_dic(train_body_dict, keyword=keyword,
                                                extracted_length=extracted_subject_length)

In [244]:
keyword_prediction_scores = {}
for mid, val_body in val_body_dict.items():
    recipient_scores = defaultdict(int)
    if (mid in val_subject_dic.keys()):
        val_subject = val_subject_dic[mid]
        mids = [mid for mid, train_subject in train_subject_dic.items() if train_subject == val_subject]
        for hit_mid in mids:
            recipients = preprocess.get_recipients(train_info, hit_mid)
            for recipient in recipients:
                recipient_scores[recipient] += 1
        keyword_prediction_scores[mid] = recipient_scores
    else:
        keyword_prediction_scores[mid] = {}
        
keyword_predictions = knntools.similar_dic_to_standard(keyword_prediction_scores)
            

In [249]:
keyword_predictions = manualtools.get_keyword_prediction(train_body_dict, val_body_dict, train_info, keyword, extracted_length=20)

In [251]:
current_score = scoring.compute_prediction_mad(keyword_predictions, val_info)
print(current_score)

0.0266901802241


In [252]:
models = [keyword_predictions, in_body_predictions]
fusion_predictions = fusiontools.reciprocal_rerank(models, 10)

In [253]:
fusion_score = scoring.compute_prediction_mad(fusion_predictions, val_info)
print(fusion_score)

0.0258267660742
